因为服务之间通过网络进行通信， 网络通信是可能产生问题的， CAP 就是处理网络问题权衡的原则

#### 一致性 Consistency
数据的分布式存储下, 数据库存在主从读写分离的操作. 写向主数据库写, 读从从数据库读. 如果主数据库写完就表示操作完成, 由于数据还未同步到从数据库, 此时从从数据库中会读不到这个数据, 用户的反应是"明明写成功了, 为什么还读不到?", 从而造成数据的不一致性. 解决方法是: 当从数据库数据同步完毕, 才表示写操作完成
一致性的实现: 
1. 由于数据存在主从同步, 写操作的响应会有一定延迟
2. 为了保证数据一致性, 会对资源暂时锁定, 待数据同步完成后才能从从数据库进行读操作
3. 如果同步失败, 读操作要返回错误信息, 不能返回未同步的旧数据

#### 可用性 Availibility
用户任何时刻的请求都能得到响应, 不会出现延迟等待或返回错误的情况  
可用性实现: 
1. 仍然保留主从同步, 但写操作不对资源锁定, 任何时刻都提供读操作服务
2. 如果从数据库还未同步完成, 可以给读操作响应旧数据, 不能返回错误信息

#### 分区容忍性 Partition tolerance
分布式系统下的每个服务因为依靠网络通信, 经常存在网络不通时, 服务之间无法进行通信. 因此不能因为网络不通就让整个分布式系统停止对外服务. 所以分区容忍性是必须满足的特性
分区容忍性实现: 
1. 使用异步方式进行主从同步, 节点之间实现松耦合
2. 添加从数据库节点, 一个从数据库挂掉, 另一个从数据库提供服务

#### CAP 之间的权衡
一致性 C 和可用性 A 之间是矛盾的. 前者要求对资源加锁, 数据一致后才能响应查询; 后者不对资源加锁, 及时数据不一致也要实时响应. 因此分布式系统只能满足其一. 从而出现了 CP 组合 和 AP 组合. (CA组合放弃了分区容忍性, 不再是分布式系统, 退化成单体数据库实现)

1. AP: 
放弃一致性, 满足可以性. 比如"订单退款", 今日退款成功, 明日查询到账. 只要用户能接受一定时间后到账即可
2. CP:   
放弃可用性, 满足一致性. 比如"跨行转账", 只有双方银行都返回操作成功后, 转账才算完成, 在此之前不响应用户操作


#### BASE 理论  
分布式一般要有限满足 AP 组合, 因为用户响应是优先考虑的. BASE 理论在 AP 的特性组合下, 通过降低可用性的程度来提供低程度的一致性 (最终一致性). 从一定程度上, 满足 CAP 同时成立的需求.
1. 可用性的降低 - 基本可用: 分布式系统在出现故障后, 只保留核心功能可用, 允许部分功能不可用. 比如: 电商网站的交易系统系统出现问题, 商品浏览功能依然满足  
2. 一致性的降低 - 最终一致性: 指系统内的数据不是时刻保持一致的, 而是经过一段时间后, 所有节点的数据达到一致. 比如: 支付的最终结果与实际交易结果一致, 但要延迟一天后显示一致
3. 增加数据的中间状态 - 软状态: 因为 BASE 理论没有满足强一致性, 当写入主节点后, 从数据库中的数据暂时处于不一致的状态, 这种状态表示数据的中间状态. 比如: 支付后显示"支付中"这个状态, 待数据最终一致后修改为"成功"状态

